In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import regularizers



In [2]:
# Load and preprocess data
data = pd.read_csv('../data/preprocessed_data.csv')
X = data.drop('Depression', axis=1)
y = data['Depression']


In [3]:

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


In [4]:

# Create an improved model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),  # Increase dropout slightly to reduce overfitting

    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),

    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],),
                 kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),

    layers.Dense(1, activation='sigmoid')
])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

In [6]:
#Compiling the model

optimizer = keras.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-5)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC()]
)


In [7]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/100
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 4s 912us/step - accuracy: 0.8167 - auc: 0.5000 - loss: nan - val_accuracy: 0.8175 - val_auc: 0.5000 - val_loss: nan
Epoch 2/100
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 3s 822us/step - accuracy: 0.8203 - auc: 0.5000 - loss: nan - val_accuracy: 0.8175 - val_auc: 0.5000 - val_loss: nan
Epoch 3/100
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 3s 841us/step - accuracy: 0.8197 - auc: 0.5000 - loss: nan - val_accuracy: 0.8175 - val_auc: 0.5000 - val_loss: nan
Epoch 4/100
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 2s 807us/step - accuracy: 0.8184 - auc: 0.5000 - loss: nan - val_accuracy: 0.8175 - val_auc: 0.5000 - val_loss: nan
Epoch 5/100
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 3s 814us/step - accuracy: 0.8174 - auc: 0.5000 - loss: nan - val_accuracy: 0.8175 - val_auc: 0.5000 - val_loss: nan
Epoch 6/100
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 3s 817us/step - accuracy: 0.8200 - auc: 0.5000 - loss: nan - val_accuracy: 0.8175 - val_auc: 0.5000 - val_loss: nan
Epoch 7/100
3075/3075 ━━━━━━━━━━━━━━━━━━━━ 3s 

In [12]:

# Evaluate the model
test_metrics = model.evaluate(X_test_scaled, y_test)
test_loss, test_accuracy, test_auc = test_metrics
print(f'\nTest accuracy: {test_accuracy:.4f}')
print(f'Test AUC: {test_auc:.4f}')

# ... rest of the saving code remains the same ...

# Save the model
import os
MODEL_DIR = os.path.join(os.getcwd(), 'saved_model')
version = 1
export_path = os.path.join(MODEL_DIR, f"{version}.keras")  # Added .keras extension

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True
)

print(f'\nModel saved to: {export_path}')

659/659 ━━━━━━━━━━━━━━━━━━━━ 0s 372us/step - accuracy: 0.8138 - auc: 0.5000 - loss: nan

Test accuracy: 0.8175
Test AUC: 0.5000


FileNotFoundError: [Errno 2] No such file or directory: '/Users/oskar/Desktop/Tartu Ülikool IT/1. kursus/1. semester/andmeteadus/Projekt/data_project/learn/saved_model/1.keras'